## Regularyzacja

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from copy import deepcopy
import csv

In [ ]:
def h_fun_multivariable(X, theta):
    '''
    :param X: ndarray postaci (n+1, m).
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: 
    '''
    return np.matmul(theta.T, X)

In [ ]:
def computeDerivativeMultivariable(X, Y, theta):
    '''
    :param X: ndarray postaci (n+1, m).
    :param y: ndarray z wartościami referencyjnymi o wymiarze (1, m)
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: wartość f. kosztu
    '''
    n,m = X.shape # number of features, number of examples
    
    diff = (h_fun_multivariable(X, theta) - Y)
    gradient = diff @ X.T
    gradient = gradient / m

    return gradient.T

In [ ]:
def calculate_cost_multivariable(X, y, theta):
    '''
    :param X: ndarray postaci (n+1, m).
    :param y: ndarray z wartościami referencyjnymi o wymiarze (1, m)
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: wartość f. kosztu
    '''
    m = y.shape[1]
    value = h_fun_multivariable(X, theta)
    return ((value - y) ** 2).sum() / (2*m)

In [ ]:
# calculate_cost_multivariable(np.array([[0, 2, 2, 0]], dtype=np.float64), np.array([[0, 0, 0, 0]], dtype=np.float64), theta = np.array([[1]], dtype=np.float64))

In [ ]:
def LinearRegression(X, Y, theta_in, learningRate, epsilon):
    i = 0
    costList = []
    derivatveList = []
    thetaList = []

    theta = deepcopy(theta_in)

    while(i < 2 or abs((costList[i-1] - costList[i-2])) > epsilon):
        derivative = computeDerivativeMultivariable(X, Y, theta)
        derivatveList.append(derivative)

        theta = theta - learningRate * derivative
        thetaList.append(theta)

        cost = calculate_cost_multivariable(X, Y, theta)
        costList.append(cost)
        
        if i < 10 or i % 100000 == 0:
            print(f"Iteration: {i}, Cost: {costList[-1]}")

        i += 1

    return [theta, i, costList]

In [ ]:
def computeResult(theta, example, X_scale, y_scale):
    return np.matmul(theta.T, example/X_scale) * y_scale

In [ ]:
def computeResultWithScaler(theta, example, x_scaler: StandardScaler):
    return np.matmul(theta.T, x_scaler.transform(example))

In [ ]:
# with open("./lin_reg_training.csv") as f:
#     csv_reader = csv.reader(f)
#     examples = []
#     for row in csv_reader:
#         examples.append([float(cell) for cell in row])
#     data = np.array(examples)
#     print(data)
#     x = data[:, 0].T
#     y = data[:, 1].T
#     x = np.reshape(x, [1, -1])
#     y = np.reshape(y, [1, -1])
#     X = np.concatenate([np.ones([1, x.shape[1]]), x])
#     x = x.T
#     y = y.T
#     X = X.T

with open('./lin_reg_training.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    data_array = np.array(data, dtype=np.float64)
    x = data_array[:, 0]
    x = np.reshape(x, [1, -1])
    X = np.concatenate([np.ones([1, x.shape[1]]), x])
    y = data_array[:, 1]
    y = np.reshape(y, [1, -1])
    x = x.T
    y = y.T
    X = X.T
    print(x)
    print(y)    

In [ ]:
print(x.shape)
print(y.shape)
print(X.shape)

In [ ]:
plt.figure()
plt.scatter(x, y)
plt.show()

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
x_train_single_dim = x
y_train = y

In [ ]:
standard_scaler = StandardScaler()
standard_scaler.fit(x_train_single_dim)
x_train_single_dim = standard_scaler.transform(x_train_single_dim)
print(x_train_single_dim)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=6)
x_train = poly_features.fit_transform(x_train_single_dim)

In [ ]:
print(x_train.shape)

In [ ]:
x_train = x_train.T
y_train = y_train.T

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
theta_in = np.zeros((x_train.shape[0], 1))
eps = 1e-8  # akceptowalna różnica dla kolejnych wartości funkcji kosztu 
alpha = 0.001  # learning rate
theta_0 = 0  # - wartości początkowe parametrów modelu
theta_1 = 0

theta, i, costList = LinearRegression(x_train, y_train, theta_in, alpha, eps)

In [ ]:
temp_x_original = np.reshape(np.linspace(10, 24, 100), [-1, 1])
temp_x = standard_scaler.transform(temp_x_original)
temp_x = poly_features.transform(temp_x)
temp_x = temp_x.T
result_ = computeResult(theta, temp_x, 1, 1)

In [ ]:
plt.figure()
plt.plot(temp_x_original, result_.T)
plt.scatter(x, y_train)
plt.show()

In [ ]:
result = computeResult(theta, x_train, 1, 1)

In [ ]:
plt.figure()
plt.plot(standard_scaler.inverse_transform(np.reshape(x_train[1, :], [-1, 1])), result.T)
plt.scatter(x, y_train)
plt.show()

In [ ]:
cost = calculate_cost_multivariable(x_train, y_train, theta)
print(cost)

In [ ]:
with open('./lin_reg_test.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    data_array = np.array(data, dtype=np.float64)
    x_test = data_array[:, 0]
    x_test = np.reshape(x_test, [1, -1])
    # X = np.concatenate([np.ones([1, x.shape[1]]), x])
    y_test = data_array[:, 1]
    y_test = np.reshape(y_test, [1, -1])
    # x_test = x_test.T
    # y_test = y_test.T
    # X = X.T
    print(x_test)
    print(y_test)

In [ ]:
x_test_normalized = standard_scaler.transform(x_test.T)
x_test_variable = poly_features.transform(x_test_normalized)

In [ ]:
cost = calculate_cost_multivariable(x_test_variable.T, y_test, theta)
print(cost)

In [ ]:
result_temp = computeResult(theta, x_test_variable.T, 1, 1)
plt.figure()
plt.scatter(x_test, y_test, )
plt.plot(temp_x_original, result_.T)
plt.show()

## Regularization

In [ ]:
def calculate_cost_multivariable_regular(X, y, theta, lambda_):
    '''
    :param X: ndarray postaci (n+1, m).
    :param y: ndarray z wartościami referencyjnymi o wymiarze (1, m)
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: wartość f. kosztu
    '''
    m = y.shape[1]
    value = h_fun_multivariable(X, theta)
    cost = ((value - y) ** 2).sum() / (2*m)
    sum_of_thetas = (theta[1:, :] ** 2).sum()
    return cost + sum_of_thetas * lambda_ / (2 * m)

In [ ]:
def computeDerivativeMultivariable(X, Y, theta):
    '''
    :param X: ndarray postaci (n+1, m).
    :param y: ndarray z wartościami referencyjnymi o wymiarze (1, m)
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: wartość f. kosztu
    '''
    n,m = X.shape # number of features, number of examples
    
    diff = (h_fun_multivariable(X, theta) - Y)
    gradient = np.matmul(diff, X.T)
    gradient = gradient / m

    return gradient.T

In [ ]:
def LinearRegression_regular(X, Y, theta_in, learningRate, epsilon, lambda_):
    n, m = X.shape # number of features, number of examples
    i = 0
    costList = []
    derivatveList = []
    thetaList = []

    theta = deepcopy(theta_in)

    while(i < 2 or abs((costList[i-1] - costList[i-2])) > epsilon):
        derivative = computeDerivativeMultivariable(X, Y, theta)
        derivatveList.append(derivative)

        theta_reg = np.concatenate([np.zeros((1, 1)), theta[1:, :]], axis=0)
        theta = theta - learningRate * (derivative + lambda_ * theta_reg) / m

        thetaList.append(theta)

        cost = calculate_cost_multivariable(X, Y, theta)
        costList.append(cost)
        
        # if i < 10 or i % 10000 == 0:
        #     print(f"Iteration: {i}, Cost: {costList[-1]}")

        i += 1

    return [theta, i, costList]

In [ ]:
def getCostRegular(x_train, y_train, x_test, y_test, lambda_):
    scaler = StandardScaler()
    scaler.fit(x_train)
    poly_features = PolynomialFeatures(degree=6)

    x_train_transformed = scaler.transform(x_train)
    x_train_poly = poly_features.fit_transform(x_train_transformed)
    x_train_poly = x_train_poly.T

    x_data = np.linspace(np.min([np.min(x_train), np.min(x_test)]), np.max([np.max(x_train), np.max(x_test)]), 100)
    x_data = np.reshape(x_data, (-1, 1))
    x_data_transformed = scaler.transform(x_data)
    x_data_poly = poly_features.transform(x_data_transformed)
    x_data_poly = x_data_poly.T

    x_test_transformed = scaler.transform(x_test)
    x_test_poly = poly_features.transform(x_test_transformed)
    x_test_poly = x_test_poly.T

    theta_in = np.zeros((x_train_poly.shape[0], 1))
    eps = 1e-5
    alpha = 0.001

    theta, i, costList = LinearRegression_regular(x_train_poly, y_train, theta_in, alpha, eps, lambda_)

    result_train = computeResult(theta, x_train_poly, 1, 1)
    result_data = computeResult(theta, x_data_poly, 1, 1)
    result_test = computeResult(theta, x_test_poly, 1, 1)

    plt.figure()
    plt.scatter(x_train.T, y_train)
    plt.plot(x_data, result_data.T)
    plt.scatter(x_test.T, y_test)
    plt.show()

    cost_training = calculate_cost_multivariable_regular(x_train_poly, y_train, theta, 0)
    cost_testing = calculate_cost_multivariable_regular(x_test_poly, y_test, theta, 0)

    return [cost_training, cost_testing]

In [ ]:
with open('./lin_reg_training.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    data_array = np.array(data, dtype=np.float64)
    x = data_array[:, 0]
    x = np.reshape(x, [1, -1])
    y = data_array[:, 1]
    y = np.reshape(y, [1, -1])
    x_train = x.T
    y_train = y
    # print(x_train)
    # print(y_train)

with open('./lin_reg_test.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    data_array = np.array(data, dtype=np.float64)
    x_test = data_array[:, 0]
    x_test = np.reshape(x_test, [1, -1])
    x_test = x_test.T
    y_test = data_array[:, 1]
    y_test = np.reshape(y_test, [1, -1])
    # print(x_test)
    # print(y_test)    

In [ ]:
lambda_list = [10, 1, 0.01, 0.001, 0.0001]
cost_list = []

for lambda_ in lambda_list:
    print(f"lambda: {lambda_}")
    cost_train, cost_test = getCostRegular(x_train, y_train, x_test, y_test, lambda_)
    cost_list.append((cost_train, cost_test))
    print(f"Train cost: {cost_train}, Test cost: {cost_test}")

In [ ]:
temp = np.array([[2], [3], [4]], dtype=np.float64)
aha = np.concatenate([np.zeros((1, 1)), temp[1:, :]], axis=0)
print(aha)

## Regresja Logistyczna

In [ ]:
def sigmoid(y):
    value_temp =  1.0 / (1.0 + np.exp(-y))
    if np.any(value_temp == 1):
        value_temp = value_temp - np.finfo(y.dtype).eps
    if np.any(value_temp == 0):
        value_temp = value_temp + np.finfo(y.dtype).eps
    return value_temp

In [ ]:
def h_fun(X, theta):
    '''
    :param X: ndarray postaci (n+1, m).
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: 
    '''
    z = np.matmul(theta.T, X)
    h = sigmoid(z)

    return h

In [ ]:
def calculate_cost(X, y, theta, lambda_):
    '''
    :param X: ndarray postaci (n+1, m).
    :param y: ndarray z wartościami referencyjnymi o wymiarze (1, m)
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: wartość f. kosztu
    '''
    m = X.shape[1]
    h = h_fun(X, theta)
    y_1 = -y*np.log(h)
    y_0 = -(1-y)*np.log(1-h)
    cost_ =  (y_1 + y_0).sum() / m
    sum_of_thetas = (theta[1:, :] ** 2).sum()
    return cost_ + sum_of_thetas * lambda_ / (2*m)

In [ ]:
def computeDerivativeMultivariable(X, Y, theta):
    '''
    :param X: ndarray postaci (n+1, m).
    :param y: ndarray z wartościami referencyjnymi o wymiarze (1, m)
    :param theta: macierz parametrów do optymalizacji postaci (n+1, 1)
    :return: wartość f. kosztu
    '''
    n,m = X.shape # number of features, number of examples
    
    diff = (h_fun(X, theta) - Y)
    gradient = np.matmul(diff, X.T)
    gradient = gradient / m

    return gradient.T

In [ ]:
def LogisticRegression(X, Y, theta_in, learningRate, epsilon, lambda_):
    n, m = X.shape # number of features, number of examples
    i = 0
    costList = []
    derivatveList = []
    thetaList = []

    theta = deepcopy(theta_in)

    while(i < 2 or abs((costList[i-1] - costList[i-2])) > epsilon):
        derivative = computeDerivativeMultivariable(X, Y, theta)
        derivatveList.append(derivative)

        theta_reg = np.concatenate([np.zeros((1, 1)), theta[1:, :]], axis=0)
        theta = theta - learningRate * (derivative + lambda_ * theta_reg) / m
        thetaList.append(theta)

        cost = calculate_cost(X, Y, theta, lambda_)
        costList.append(cost)
        
        # if(i < 10 or i % 100000 == 0):
        #     print(f"Iteration: {i}, Cost: {costList[-1]}")

        i += 1

    return [theta, i, costList]

In [ ]:
def computeResult(theta, example, X_scale, y_scale):
    return sigmoid(np.matmul(theta.T, example/X_scale) * y_scale)

In [ ]:
with open('./logi_reg_training.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    data_array = np.array(data, dtype=np.float64)
    x = data_array[:, 0:2]
    y = data_array[:, 2]
    x_train = x
    y_train = y

with open('./logi_reg_test.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    data_array = np.array(data, dtype=np.float64)
    x_test = data_array[:, 0:2]
    y_test = data_array[:, 2]
    x_test = x_test

In [ ]:
def getCostRegular(x_train, y_train, x_test, y_test, lambda_):
    scaler = StandardScaler()
    scaler.fit(x_train)
    poly_features = PolynomialFeatures(degree=6)

    x_train_transformed = scaler.transform(x_train)
    x_train_poly = poly_features.fit_transform(x_train_transformed)
    x_train_poly = x_train_poly.T

    # x_data = np.linspace(np.min([np.min(x_train), np.min(x_test)]), np.max([np.max(x_train), np.max(x_test)]), 100)
    # x_data = np.reshape(x_data, (-1, 1))
    # x_data_transformed = scaler.transform(x_data)
    # x_data_poly = poly_features.transform(x_data_transformed)
    # x_data_poly = x_data_poly.T

    x_test_transformed = scaler.transform(x_test)
    x_test_poly = poly_features.transform(x_test_transformed)
    x_test_poly = x_test_poly.T

    theta_in = np.zeros((x_train_poly.shape[0], 1))
    eps = 1e-7
    alpha = 0.001

    theta, i, costList = LogisticRegression(x_train_poly, y_train, theta_in, alpha, eps, lambda_)

    result_train = computeResult(theta, x_train_poly, 1, 1)
    # result_data = computeResult(theta, x_data_poly, 1, 1)
    result_test = computeResult(theta, x_test_poly, 1, 1)
    
    # logistic regression
    value0_train = x_train[np.where(y_train == 0), :]
    value1_train = x_train[np.where(y_train == 1), :]
    value0_train = np.reshape(value0_train, (-1, 2))
    value1_train = np.reshape(value1_train, (-1, 2))
    
    value0_test = x_test[np.where(y_test == 0), :]
    value1_test = x_test[np.where(y_test == 1), :]
    value0_test = np.reshape(value0_test, (-1, 2))
    value1_test = np.reshape(value1_test, (-1, 2))
    
    ax = plt.figure()
    plt.scatter(value0_train[:, 0], value0_train[:, 1], marker='o')
    plt.scatter(value1_train[:, 0], value1_train[:, 1], marker='x')

    plt.scatter(value0_test[:, 0], value0_test[:, 1], marker='.')
    plt.scatter(value1_test[:, 0], value1_test[:, 1], marker='*')
    
    x1 = x_train[:, 0]
    x2 = x_train[:, 1]
    x1_range = np.linspace(x1.min(), x1.max(), 100)
    x2_range = np.linspace(x2.min(), x2.max(), 100)
    x1x1, x2x2 = np.meshgrid(x1_range, x2_range)
    points = np.c_[x1x1.ravel(), x2x2.ravel()]
    x1x2_range_comb = scaler.transform(points)
    x1x2_range_comb = poly_features.transform(x1x2_range_comb)
    X_range_comb = x1x2_range_comb.T
    h_range_comb = h_fun(X_range_comb, theta)
    h_range_comb.shape = x1x1.shape
    plt.contour(x1x1, x2x2, h_range_comb, levels=[0.5], colors='g')
    
    ax.legend(["does not belong to a class - train", "belongs to a class - training", "does not belong to a class - test", "belong to a class - test"])
    
    plt.show()
    
    # plt.figure()
    # plt.scatter(x_train.T, y_train)
    # plt.plot(x_data, result_data.T)
    # plt.scatter(x_test.T, y_test)
    # plt.show()

    cost_training = calculate_cost(x_train_poly, y_train, theta, 0)
    cost_testing = calculate_cost(x_test_poly, y_test, theta, 0)

    return [cost_training, cost_testing]

In [ ]:
lambda_ = [10, 1, 0.5, 0.1, 0.05, 0.01, 0]
cost_list = []

for lambda_ in lambda_list:
    print(f"lambda: {lambda_}")
    cost_train, cost_test = getCostRegular(x_train, y_train, x_test, y_test, lambda_)
    cost_list.append((cost_train, cost_test))
    print(f"Train cost: {cost_train}, Test cost: {cost_test}")

## Logistic Regression Scikit Learn

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
 


In [ ]:
data = load_iris()
x = data.data
y = data.target
 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=12345)
standard_scaler = StandardScaler()
x_train = standard_scaler.fit_transform(x_train)
x_test = standard_scaler.transform(x_test)

In [ ]:
log_regr = LogisticRegression(penalty=None)
log_regr.fit(x_train, y_train)
y_train_probab = log_regr.predict_proba(x_train)
y_test_probabs = log_regr.predict_proba(x_test)
cost_train = log_loss(y_train, y_train_probab)
cost_test = log_loss(y_test, y_test_probabs)
y_test_pred = log_regr.predict(x_test)
report = classification_report(y_test, y_test_pred)
print(f"Train cost: {cost_train}")
print(f"Test cost: {cost_test}")
print(report)

In [ ]:
data = load_iris()
x = data.data
y = data.target
 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=12345)
standard_scaler = StandardScaler()
x_train = standard_scaler.fit_transform(x_train)
x_test = standard_scaler.transform(x_test)

In [ ]:
poly_features = PolynomialFeatures(degree=6)
x_train = poly_features.fit_transform(x_train)
x_test = poly_features.transform(x_test)

log_regr = LogisticRegression(penalty=None)
log_regr.fit(x_train, y_train)
y_train_probab = log_regr.predict_proba(x_train)
y_test_probabs = log_regr.predict_proba(x_test)
cost_train = log_loss(y_train, y_train_probab)
cost_test = log_loss(y_test, y_test_probabs)
y_test_pred = log_regr.predict(x_test)
report = classification_report(y_test, y_test_pred)
print(f"Train cost: {cost_train}")
print(f"Test cost: {cost_test}")
print(report)

In [ ]:
lambda_list = [100, 10, 1, 0.1, 0.01]

for lambda_ in lambda_list:
    data = load_iris()
    x = data.data
    y = data.target
     
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=12345)
    standard_scaler = StandardScaler()
    x_train = standard_scaler.fit_transform(x_train)
    x_test = standard_scaler.transform(x_test)
    
    poly_features = PolynomialFeatures(degree=6)
    x_train = poly_features.fit_transform(x_train)
    x_test = poly_features.transform(x_test)
    
    log_regr = LogisticRegression(penalty="l2", n_jobs = -1, C = lambda_, max_iter = 10000)
    log_regr.fit(x_train, y_train)
    y_train_probab = log_regr.predict_proba(x_train)
    y_test_probabs = log_regr.predict_proba(x_test)
    cost_train = log_loss(y_train, y_train_probab)
    cost_test = log_loss(y_test, y_test_probabs)
    y_test_pred = log_regr.predict(x_test)
    report = classification_report(y_test, y_test_pred)
    print(f"lambda: {lambda_}")
    print(f"Train cost: {cost_train}")
    print(f"Test cost: {cost_test}")
    print(report)